In [1]:
!pip install transformers
!pip install datasets
!pip install seqeval
!pip install ipywidgets
!pip install IProgress
!pip install pytorch-crf

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
import torch

In [44]:
import transformers
from datasets import load_dataset, load_metric
from datasets import Sequence, ClassLabel
from datasets import Value

import bert_crf_models as crf_models

In [64]:
import pickle
from transformers.data.data_collator import DataCollatorWithPadding

In [48]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_name = "bert-base-uncased"
batch_size = 16

In [5]:
# dataset = load_dataset("conll2003", revision="master")

In [6]:
# dataset

In [11]:
PE_DATA_FILE = '/notebooks/token_classification/data/pe_dataset_task_1_bio_only.pt'

In [15]:
with open(PE_DATA_FILE, "rb") as f:
    pe_dataset = pickle.load(f)
    

In [16]:
pe_dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 1091
    })
    test: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 355
    })
    validation: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 273
    })
})

In [17]:
pe_dataset

DatasetDict({
    train: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 1091
    })
    test: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 355
    })
    validation: Dataset({
        features: ['split', 'essay', 'essay_nr', 'para_nr', 'paragraph', 'paragraph_words', 'paragraph_tags', 'tags_list_int', 'paragraph_tags_bio_only', 'paragraph_tags_bio_only_int', 'paragraph_tags_bio_only_correction'],
        num_rows: 273
    })
})

In [18]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'tags_list_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_correction': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [19]:
pe_dataset = pe_dataset.rename_column("tags_list_int", "paragraph_tags_list_int")

In [20]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_list_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_correction': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [21]:
pe_dataset = pe_dataset.rename_column("paragraph_tags_list_int", "paragraph_tags_int")

In [22]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_correction': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

### my work today

In [23]:
# what if we sequence the int labels to the class labels? will it work?

In [24]:
pe_dataset['train'].features['paragraph_tags_bio_only_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [25]:
pe_dataset['train'].features['paragraph_tags_bio_only_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [26]:
pe_dataset['test'].features['paragraph_tags_bio_only_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [27]:
pe_dataset['validation'].features['paragraph_tags_bio_only_int'] = Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None),
                                             length=-1, id=None)

In [28]:
pe_dataset['test'].features['paragraph_tags_bio_only_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [29]:
pe_dataset['validation'].features['paragraph_tags_bio_only_int']

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [30]:
pe_dataset['train'].features

{'split': Value(dtype='string', id=None),
 'essay': Value(dtype='string', id=None),
 'essay_nr': Value(dtype='string', id=None),
 'para_nr': Value(dtype='int64', id=None),
 'paragraph': Value(dtype='string', id=None),
 'paragraph_words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_int': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
 'paragraph_tags_bio_only': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'paragraph_tags_bio_only_int': Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None),
 'paragraph_tags_bio_only_correction': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [68]:
pe_dataset["train"][0]["paragraph_tags_bio_only"]

['O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'I',
 'O']

In [32]:
pe_dataset["train"].features["paragraph_tags_bio_only_int"]

Sequence(feature=ClassLabel(num_classes=3, names=['O', 'B-COMP', 'I-COMP'], names_file=None, id=None), length=-1, id=None)

In [33]:
# seems correct

In [34]:
from transformers import AutoTokenizer

In [73]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmplew7bz8f


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializ

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpiaw354ig


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

storing https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://hugg

In [69]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [70]:
def tokenize_and_align_labels_hf(examples):
    tokenized_inputs = tokenizer(
        examples["paragraph_words"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["paragraph_tags_bio_only"]
    #print(examples["paragraph_tags"])
    #all_labels = labels_object.str2int(examples["paragraph_tags"])
    
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [71]:
label_all_tokens = True

In [72]:
tokenized_datasets = pe_dataset.map(tokenize_and_align_labels_hf, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

TypeError: not all arguments converted during string formatting

In [74]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["paragraph_words"], 
                                 truncation=True, 
                                 padding=True, 
                                 max_length=512, 
                                 is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["paragraph_tags_bio_only"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:                            # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:              # Only label the first token of a given word.
                label_ids.append(label[word_idx])
                previous_word_idx = word_idx # *** modif ***
            # *** modif ***
            elif word_idx == previous_word_idx:              # Only label the first token of a given word.
                label_ids.append(-100)
                previous_word_idx = word_idx
            # *** end modif ***

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


pe_dataset = pe_dataset.map(tokenize_and_align_labels, batched=True)


  0%|          | 0/2 [00:00<?, ?ba/s]

ArrowInvalid: Could not convert 'O' with type str: tried to convert to int64

In [40]:
label_list = pe_dataset["train"].features['paragraph_tags_bio_only_int'].feature.names

In [41]:
label_list

['O', 'B-COMP', 'I-COMP']

In [42]:
pe_dataset["train"].features["paragraph_tags_bio_only_int"].feature.names

['O', 'B-COMP', 'I-COMP']

In [49]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

#model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

model = crf_models.BERT_CRF.from_pretrained(model_name, num_labels=len(label_list))

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BERT_CRF: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BERT_CRF from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BERT_CRF from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BERT_CRF were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['crf.end_transitions', 'classifier.bi

In [50]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BERT_CRF(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
   

In [59]:
RESULTS_FOLDER = "/notebooks/bert_crf/results"
NB_EPOCHS = 32

In [66]:
data_collator = DataCollatorForTokenClassification(tokenizer)


# if args.model == 'bert':
#     model = BERT_CRF.from_pretrained(MODEL_NAME, num_labels=num_labels)
# else:
#     model = DistilBERT_CRF.from_pretrained(MODEL_NAME, num_labels=num_labels)
# model.to(device)


# if not FINETUNE_MODE:

#     # Freeze weights of the 'bert' part of the model
#     if args.model == 'bert':
#         for param in model.bert.parameters():
#             param.requires_grad = False
#     elif args.model == 'distilbert':
#         for param in model.distilbert.parameters():
#             param.requires_grad = False
        
#     # Unfreeze weights of the 'classifier' part of the model (just ot be sure)
#     for param in model.classifier.parameters():
#         param.requires_grad = True


training_args = TrainingArguments(
    # output
    output_dir=RESULTS_FOLDER,          
    
    # params
    num_train_epochs=NB_EPOCHS,               # nb of epochs
    per_device_train_batch_size=batch_size,   # batch size per device during training
    per_device_eval_batch_size=batch_size,    # cf. paper Sun et al.
    learning_rate=1e-3,                         # (cf. paper Sun et al.)
#     warmup_steps=500,                       # number of warmup steps for learning rate scheduler
    warmup_ratio=0.1,                         # cf. paper Sun et al.
    weight_decay=0.01,                        # strength of weight decay
    # eval
    evaluation_strategy="steps",              # cf. paper Sun et al.
    eval_steps=50,               # cf. paper Sun et al.
    # log
    logging_dir=RESULTS_FOLDER,#+f'logs-{args.model}_finetune_{FINETUNE_MODE}',  
    logging_strategy='steps',
    logging_steps=50,
    # save
    save_strategy='steps',
    save_total_limit=2,
    # save_steps=20, # default 500
    load_best_model_at_end=True,              # cf. paper Sun et al.
    metric_for_best_model='eval_loss'
    # metric_for_best_model='f1'
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=pe_dataset['train'],
    eval_dataset=pe_dataset['validation'],
    data_collator=data_collator,
    # compute_metrics=compute_metrics
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)





PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [67]:
results = trainer.train()

training_time = results.metrics["train_runtime"]

# save best model
# trainer.save_model(os.path.join(RESULTS_FOLDER, f'checkpoint-best-{args.model}_finetune_{FINETUNE_MODE}'))



The following columns in the training set  don't have a corresponding argument in `BERT_CRF.forward` and have been ignored: split, essay_nr, paragraph, paragraph_tags_bio_only, paragraph_tags, para_nr, paragraph_words, paragraph_tags_bio_only_int, essay, paragraph_tags_int, paragraph_tags_bio_only_correction.
***** Running training *****
  Num examples = 1091
  Num Epochs = 32
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2208


ZeroDivisionError: integer division or modulo by zero

In [ ]:
# ======= #
# Results #
# ======= #


metric = load_metric("seqeval")

# # load model
# model_file = os.path.join(RESULTS_FOLDER, f'checkpoint-best-{args.model}_finetune_{FINETUNE_MODE}')
# model = AutoModelForTokenClassification.from_pretrained(model_file, num_labels=num_labels)
# model.to(device)

model.eval()

label_list = label_list

test_trainer = Trainer(model, data_collator=DataCollatorWithPadding(tokenizer))
test_preds, test_labels, _ = test_trainer.predict(pe_dataset['test'])
# test_preds = np.argmax(test_raw_preds, axis=2)


true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(test_preds, test_labels)
]

true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(test_preds, test_labels)
]

test_results = metric.compute(predictions=true_predictions, references=true_labels)

results = (test_results, training_time)

with open(os.path.join(RESULTS_FOLDER, RESULTS_FILE), 'wb') as fh:
    pickle.dump(results, fh)


In [51]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=32,
    weight_decay=0.01,
    eval_steps=20,
    logging_steps=20
    #push_to_hub=True,
)

In [52]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [53]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [54]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [55]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [56]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BERT_CRF.forward` and have been ignored: split, essay_nr, paragraph, paragraph_tags_bio_only, paragraph_tags, para_nr, paragraph_words, paragraph_tags_bio_only_int, essay, paragraph_tags_int, paragraph_tags_bio_only_correction.
***** Running training *****
  Num examples = 1091
  Num Epochs = 32
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2208
/opt/conda/lib/python3.8/site-packages/torchcrf/__init__.py:249: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at  /opt/pytorch/pytorch/aten/src/ATen/native/TensorCompare.cpp:328.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


Epoch,Training Loss,Validation Loss


The following columns in the evaluation set  don't have a corresponding argument in `BERT_CRF.forward` and have been ignored: split, essay_nr, paragraph, paragraph_tags_bio_only, paragraph_tags, para_nr, paragraph_words, paragraph_tags_bio_only_int, essay, paragraph_tags_int, paragraph_tags_bio_only_correction.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16


AxisError: axis 2 is out of bounds for array of dimension 2

In [44]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: essay_nr, paragraph, paragraph_tags_int, paragraph_tags_bio_only_correction, paragraph_tags, essay, para_nr, paragraph_tags_bio_only_int, paragraph_tags_bio_only, split, paragraph_words.
***** Running Evaluation *****
  Num examples = 273
  Batch size = 16


{'eval_loss': 0.47586920857429504,
 'eval_precision': 0.680320569902048,
 'eval_recall': 0.7966631908237748,
 'eval_f1': 0.7339097022094139,
 'eval_accuracy': 0.9174831460674158,
 'eval_runtime': 1.3818,
 'eval_samples_per_second': 197.573,
 'eval_steps_per_second': 13.027,
 'epoch': 32.0}

In [45]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: essay_nr, paragraph, paragraph_tags_int, paragraph_tags_bio_only_correction, paragraph_tags, essay, para_nr, paragraph_tags_bio_only_int, paragraph_tags_bio_only, split, paragraph_words.
***** Running Prediction *****
  Num examples = 355
  Batch size = 16


{'COMP': {'precision': 0.7089655172413794,
  'recall': 0.8184713375796179,
  'f1': 0.7597930524759793,
  'number': 1256},
 'overall_precision': 0.7089655172413794,
 'overall_recall': 0.8184713375796179,
 'overall_f1': 0.7597930524759793,
 'overall_accuracy': 0.9218603030830865}

In [46]:
# task: try to do only with the BIO. then see the results.

In [47]:
predictions, labels, _ = trainer.predict(tokenized_datasets["train"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results_train = metric.compute(predictions=true_predictions, references=true_labels)
results_train

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: essay_nr, paragraph, paragraph_tags_int, paragraph_tags_bio_only_correction, paragraph_tags, essay, para_nr, paragraph_tags_bio_only_int, paragraph_tags_bio_only, split, paragraph_words.
***** Running Prediction *****
  Num examples = 1091
  Batch size = 16


{'COMP': {'precision': 0.9798994974874372,
  'recall': 0.9893190921228304,
  'f1': 0.9845867658782886,
  'number': 3745},
 'overall_precision': 0.9798994974874372,
 'overall_recall': 0.9893190921228304,
 'overall_f1': 0.9845867658782886,
 'overall_accuracy': 0.9989079928239528}